In [128]:
import pandas as pd
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
from gensim import utils

In [129]:
df = pd.read_excel('lab2 (Task 2).xlsx')
df.head()

,HYPONYM,HYPERNYM,SNIPPET,URL
0,ПОЛУМАРАФОН,СОРЕВНОВАНИЕ,"Полумарафон — дисциплина лёгкой атлетики, пред...",https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%...
1,ПОЛУМАРАФОН,СОСТЯЗАНИЕ,Полумарафон — спорт. состязание в беге на дист...,https://kartaslov.ru/%D0%B7%D0%BD%D0%B0%D1%87%...
2,ПОЛУМАРАФОН,ЛЕГКОАТЛЕТИЧЕСКАЯ БЕГОВАЯ ДИСЦИПЛИНА,"Полумарафон — дисциплина лёгкой атлетики, пред...",https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%...
3,ПОЛУМАРАФОН,ЗАБЕГ,"Полумарафон — дисциплина лёгкой атлетики, пред...",https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%...
4,ПОЛУМАРАФОН,ДИСЦИПЛИНА ЛЁГКОЙ АТЛЕТИКИ,"Полумарафон — дисциплина лёгкой атлетики, пред...",https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%...


In [130]:
START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [131]:
def get_hyperonyms(row):
    main_word = row['HYPONYM']
    text = row['SNIPPET']
    
    HYPONYM = eq(utils.deaccent(main_word.lower()))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(HYPONYM, MID, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM),
        rule(HYPONYM, END),
        rule(END, MID, END),
        rule(END, MID, MID, END),
    )
    parser = Parser(RULE) 
    text = utils.deaccent(text)
    # print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('', '')
    hyps = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hyps.append(k)
    print(main_word, hyps)
    return hyps

In [132]:
df['dec_hyperonyms'] = df.apply(lambda row: get_hyperonyms(row), axis=1)

ПОЛУМАРАФОН [['полумарафон', '—', 'дисциплина', 'легкои', 'атлетики'], ['меньшую', 'марафонскои', '—', '21', 'км']]
ПОЛУМАРАФОН [['полумарафон', '—', 'спорт']]
ПОЛУМАРАФОН [['полумарафон', '—', 'дисциплина', 'легкои', 'атлетики'], ['меньшую', 'марафонскои', '—', '21', 'км']]
ПОЛУМАРАФОН [['полумарафон', '—', 'дисциплина', 'легкои', 'атлетики'], ['меньшую', 'марафонскои', '—', '21', 'км']]
ПОЛУМАРАФОН [['полумарафон', '—', 'дисциплина', 'легкои', 'атлетики'], ['меньшую', 'марафонскои', '—', '21', 'км']]
ПОЛУМАСКА []
ПОЛУМАСКА []
ПОЛУМАСКА [['полумаска', '—', 'узкая', 'маска']]
ПОЛУМАСКА [['часть', 'сизод', '—', 'часть', 'сизод']]
ПОЛУМАСКА [['полумаска', '—', 'маска']]
ПОЛУПАЛЬТО [['полупальто', 'в', 'элегантном', 'стиле', 'одежды']]
ПОЛУПАЛЬТО [['мама', 'в', 'это', 'время']]
ПОЛУПАЛЬТО [['полупальто', '—', 'короткое', 'пальто']]
ПОЛУПАЛЬТО [['полупальто', 'таких', 'проблем']]
ПОЛУПАЛЬТО [['полупальто', '—', 'короткое', 'пальто', 'особого', 'фасона']]
ПОЛУПОДВАЛ [['полуподвал', '—', 'эт